https://assaeunji.github.io/machine%20learning/2020-11-29-implicitfeedback/

# test 3주 정합성 검증
- 

# 추천 모델
- ALS MF, LMF, MP (총 3개)
- 총 3개의 추천을 진행하며 MF와 LMF 의 경우 콜드스타트 유저(신규 유저)인 경우 MP로 추천 진행

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [11]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 621553.01it/s]


In [12]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [13]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

df['date_paid'] = pd.to_datetime(df['date_paid'])
# 3개월 전 날짜 확인
df['date_paid'].max()-relativedelta(months=3)

Timestamp('2022-06-13 08:59:21.151000+0000', tz='UTC')

In [14]:
# date_state = "2022-06-13"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
df_date = df_only_paid.copy()
# 전체 데이터를 가져오기 때문에 전처리 진행 안함
# df_date = df_only_paid[df_only_paid['date_paid'] >= date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

# 도서, 소모품 카테고리
# df_book = complete_df[complete_df['name'].isin(['도서','소모품'])].sort_values(by='date_paid')

In [15]:
# none 값 확인하기
df_book.isna().sum()

_id                 0
date_created        0
regular_price       0
sale_price          0
three_months        0
date_paid           0
customer_id         0
paid                0
name_x              0
category_id_y       0
product_ids         0
quantity            0
price               0
price_total         0
age_group        3007
한의사 여부             79
사업자 여부             79
cancelled           0
name                0
slug                0
dtype: int64

## 전체 데이터 EDA

In [16]:
print('중복 제거 전:',len(only_book), '중복 제거 후:',len(df_book))

중복 제거 전: 38395 중복 제거 후: 37866


In [17]:
print('전체 데이터 수:',len(df_book))

전체 데이터 수: 37866


In [18]:
print('아이템 수:',len(df_book.product_ids.unique()),'유저 수:',len(df_book.customer_id.unique()))

아이템 수: 342 유저 수: 7410


# 2.train test split
- 마지막 3주 분량을 test로 선정합니다.

In [19]:
from datetime import datetime, timedelta

# 마지막 날 시간이 짧으므로 이전 날부터 하루 씩 test 분할
df_book['date_paid'].max()

Timestamp('2022-09-13 08:51:40+0000', tz='UTC')

In [20]:
# 마지막 날 기준으로 test 전처리
datetime(2022,9,13)-timedelta(days=2)

datetime.datetime(2022, 9, 11, 0, 0)

In [21]:
# train test 분리
date = '2022-09-11'
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'].dt.date == pd.to_datetime(date)]

In [22]:
# test 날짜 확인
print('첫날 :',test_before_preprocess['date_paid'].min(),'마지막 날:',test_before_preprocess['date_paid'].max())
print('test 총 기간:',test_before_preprocess['date_paid'].max()-test_before_preprocess['date_paid'].min())

첫날 : 2022-09-11 13:03:15.786000+00:00 마지막 날: 2022-09-11 20:17:23.892000+00:00
test 총 기간: 0 days 07:14:08.106000


## 전체 아이템 중복 확인

In [23]:
# product_ids, name_x 수는 일치
len(df_book.product_ids.unique()), len(df_book.name_x.unique())

(342, 370)

In [24]:
# 중복 제거 후 수 비교 확인
# 252로 일치하여 문제 없음
len(df_book.drop_duplicates(subset=['product_ids','name_x']).name_x.unique())

370

## train test 아이템 중복 확인

In [25]:
len(train_before_preprocess.product_ids.unique()),len(test_before_preprocess.product_ids.unique())

(341, 4)

In [26]:
len(set(train_before_preprocess.product_ids.unique())-set(test_before_preprocess.product_ids.unique()))

337

In [27]:
# test 아이템에 train 없는 아이템 확인
len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique()))

0

In [28]:
# test 만 있는 item 제거
only_test_items = set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())
if_prepro_test = test_before_preprocess[~test_before_preprocess['product_ids'].isin(only_test_items)]

In [29]:
test = test_before_preprocess.copy()

In [30]:
# train 변수 명 변경
train = train_before_preprocess.copy()

In [31]:
# test 전처리 진행했을 경우
print('원본 test 수:', len(test))
print('전처리 진행했을 경우 test 수:', len(if_prepro_test))

원본 test 수: 4
전처리 진행했을 경우 test 수: 4


# train test eda

### 전처리 전후 비교

In [32]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 37831 train 전처리 후: 37831


In [33]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 4 test 전처리 후: 4


### user 수 비교 

In [34]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 7406


In [35]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 2


In [36]:
# 신규 유저는 MP 같은 다른 방법으로 추천 진행해야 함
print('test 만 있는 신규 유저 :',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))

test 만 있는 신규 유저 : 0


### item 개수 비교

In [37]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 341 test 아이템 수 : 4


In [38]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 337


In [39]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 0


# 3. sparse matrix 만들기

## ALS MF Matrix

In [40]:
PdIds = train.product_ids.unique()

PdIdToIndex = {}
indexToPdId = {}

colIdx = 0

for PdId in PdIds:
    PdIdToIndex[PdId] = colIdx
    indexToPdId[colIdx] = PdId
    colIdx += 1
    
userIds = train.customer_id.unique()

userIdToIndex = {}
indexToUserId = {}

rowIdx = 0

for userId in userIds:
    userIdToIndex[userId] = rowIdx
    indexToUserId[rowIdx] = userId
    rowIdx += 1

import scipy.sparse as sp

rows = []
cols = []
vals = []

for row in train.itertuples():
    rows.append(userIdToIndex[row.customer_id])
    cols.append(PdIdToIndex[row.product_ids])
    vals.append(1)

purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

matrix = purchase_sparse.todense()
matrix

matrix([[1, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

### Most_popular_matrix

In [41]:
most_popular = train.groupby(['product_ids','name_x']).count()['customer_id'].reset_index()
most_popular

,product_ids,name_x,customer_id
0,5d13115e32026c0b35383897,KCD 한방내과 진찰진단 가이드라인,804
1,5d132e964b25b80d9fb1f352,노인요양병원 진료지침서,63
2,5d5373e94e77525ec5ca1135,통증 치료를 위한 근육 초음파와 주사 테크닉,7
3,5d5373e94e77525ec5ca1135,통증치료를 위한 근육 초음파와 주사 테크닉,268
4,5d59ee854e77525ec5ca1212,일차진료 한의사를 위한 보험한약입문 - 둘째 판,517
...,...,...,...
366,62da5dd81b3c480022189fdf,외래에서 꼭 알아야 할 통증증후군 137가지,149
367,62ff2f43ca740b0024397ca4,초음파 유도하 침 시술 가이드북,354
368,63046e9358e1680033a580d6,영어 진료 가이드북,225
369,630dbd6f68554000236d9fda,(증보판)사상의학의 원리와 방제,1


### Medistream_prediction_matrix
- 메디스트림 메디마켓에서 제공하는 정렬 추천 성능 비교를 위한 df 구현
- 인기도순, 최신순, 과거순, 높은 가격순, 낮은 가격순, 이름순 (총 6 가지)
- 각각 구현해보고 학습 모델 대비 성능 비교

In [42]:
medistream_prediction_df = train[['date_created','regular_price','sale_price','three_months','product_ids','name_x']]
medistream_prediction_preprop_df = medistream_prediction_df.drop_duplicates(subset=['product_ids'], ignore_index=True)
medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])
# sale_prices가 0이면 regular_price 값으로 채워넣어야하는데 0이 없음(전처리 필요 무)

<ipython-input-42-4d4e36d189e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])


# Sparsity 확인

In [43]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

98.50200717021865

# 4. Model

## Model 학습 진행
- real test 만들기
- implict 라이브러리 사용(MF,LMF)
- MF 구현 모델 사용

In [44]:
# real test 
ground_trues = []
for user_id in test['customer_id'].unique():
    ground_trues.append({'id': user_id,\
    'items':list(test[test['customer_id']==user_id].product_ids)
    })

## ALS fit

In [45]:
als_model = ALS(factors=20, regularization=0.01, iterations = 50, random_state=42)
als_model.fit(purchase_sparse)

  0%|          | 0/50 [00:00<?, ?it/s]

In [46]:
# item, user vector 추출
als_item_factors = als_model.item_factors
als_user_factors = als_model.user_factors

In [47]:
# 각 shape 확인
als_item_factors.shape, als_user_factors.shape

((341, 20), (7406, 20))

## LMF fit

In [48]:
from implicit.lmf import LogisticMatrixFactorization as LMF

In [49]:
lmf_model = LMF(factors=20, regularization=0.001, iterations = 20, random_state=42)
lmf_model.fit(purchase_sparse)

  0%|          | 0/20 [00:00<?, ?it/s]

In [50]:
lmf_item_factors = lmf_model.item_factors
lmf_user_factors = lmf_model.user_factors

In [51]:
lmf_user_factors.shape, lmf_item_factors.shape

((7406, 22), (341, 22))

# 5. prediction

# ALS mf prediction

In [52]:
# 신규 유저인 경우 mp로 넣기
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
als_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(als_predict_list):
    als_predict_list[idx]['items'] = pred_list['items'][:100]

# LMF prediction

In [53]:
# 신규 유저 mp로 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값
lmf_predict_list = []
for user_id in test['customer_id'].unique():
    try:
        result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
        lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
    except:
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})
        
# 100 개만 예측하기
for idx, pred_list in enumerate(lmf_predict_list):
    lmf_predict_list[idx]['items'] = pred_list['items'][:100]

# most popular prediction

In [54]:
# 전체 도서에 대한 판매 만큼 정렬 후 넣기
most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

# test 예측값, 이미 구매 했을 경우 제외
predict_popular_list = []
for user_id in test['customer_id'].unique():
    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
    predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                            ]})

# 100 개만 예측하기
for idx, pred_list in enumerate(predict_popular_list):
    predict_popular_list[idx]['items'] = pred_list['items'][:100]

# medistream prediction
- 메디스트림 메디마켓에서 제공하는 정렬 추천 성능 비교
- 인기도순, 최신순, 과거순, 높은 가격순, 낮은 가격순, 이름순 (총 6 가지)
- 각각 구현해보고 학습 모델 대비 성능 비교

In [55]:
# 인기도순
medistream_popular_list = medistream_prediction_preprop_df.sort_values(by='three_months', ascending=False).index
# 최신순
medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index
# 오랜된 순
medistream_oldest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=True).index
# 높은 가격 순
medistream_high_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=False).index
# 낮은 가격 순
medistream_low_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=True).index
# 이름 순
medistream_name_sort_list = medistream_prediction_preprop_df.sort_values(by='name_x',ascending=True).index

def medistream_prediction_method(predict_num:int ,medi_predict_list:list)->list:
    medistream_predict_list = []
    for user_id in test['customer_id'].unique():
        medistream_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] \
                                                                       for num in medi_predict_list]})

    # 100 개만 예측하기
    for idx, pred_list in enumerate(medistream_predict_list):
        medistream_predict_list[idx]['items'] = pred_list['items'][:predict_num]
        
    return medistream_predict_list

In [56]:
medistream_predict_popular_list = medistream_prediction_method(100, medistream_popular_list)
medistream_predict_latest_list = medistream_prediction_method(100, medistream_latest_list)
medistream_predict_oldest_list = medistream_prediction_method(100, medistream_oldest_list)
medistream_predict_high_price_list = medistream_prediction_method(100, medistream_high_price_list)
medistream_predict_low_price_list = medistream_prediction_method(100, medistream_low_price_list)
medistream_predict_name_sort_list = medistream_prediction_method(100, medistream_name_sort_list)

# 6. evaluation

## NDCG 평가지표

In [57]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)


        return ndcg_score

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score = self._eval(gt_list, rec_list)
            print(f"nDCG: {ndcg_score:.6}")
        except Exception as e:
            print(e)


# ALS NDCG

In [58]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, als_predict_list)

nDCG: 0.048222


In [59]:
len(als_predict_list),len(ground_trues)

(2, 2)

In [60]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, als_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

1

# LMF NDCG

In [61]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, lmf_predict_list)

nDCG: 0.0992236


In [62]:
len(lmf_predict_list),len(ground_trues)

(2, 2)

In [63]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, lmf_predict_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

2

# most popular NDCG

In [64]:
# most popular
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, predict_popular_list)

nDCG: 0.425172


In [65]:
len(predict_popular_list),len(ground_trues)

(2, 2)

In [66]:
# 아이템 맞춘 개수
cnt = 0
for gt, pred_list in zip(ground_trues, predict_popular_list):
    for pred in pred_list['items']:
        if pred in gt['items']:
            cnt += 1
cnt

2

## medistream prediction NDCG

In [67]:
def medistream_prediction(ground_trues:list, predict_list:list):
    evaluator = CustomEvaluator()
    ndcg = evaluator._eval(ground_trues, predict_list)
    
    assert len(predict_list) == len(ground_trues)
    
    cnt = 0
    for gt, pred_list in zip(ground_trues, predict_list):
        for pred in pred_list['items']:
            if pred in gt['items']:
                cnt += 1
    return ndcg, cnt

In [68]:
medistream_predict_score = {'medistream_predict':['medi_popular','latest','oldest','higt_price','low_price','name_sort'], \
                            'ndcg':[], 'cnt':[]}

medistream_predict_list = [medistream_predict_popular_list, medistream_predict_latest_list, medistream_predict_oldest_list,\
                          medistream_predict_high_price_list, medistream_predict_low_price_list, medistream_predict_name_sort_list]

for medistream_predict in medistream_predict_list:
    ndcg, cnt = medistream_prediction(ground_trues, medistream_predict)
    medistream_predict_score['ndcg'].append(ndcg)
    medistream_predict_score['cnt'].append(cnt)
pd.DataFrame(medistream_predict_score)    

,medistream_predict,ndcg,cnt
0,medi_popular,0.425172,2
1,latest,0.082848,1
2,oldest,0.000000,0
3,higt_price,0.193426,1
4,low_price,0.072170,1
5,name_sort,0.058004,1


# 7. hyper parameter tuning

## 7-1. ALS MF hypter parameter tuning

In [69]:
als_mf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[]}

factors = [5,10,15,20]
regularizations = [0.01,0.005]
iterations = [5,10,15,20,25,30,40,50]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
            als_model.fit(purchase_sparse, show_progress=False)
            
            # 신규 유저인 경우 mp로 넣기
            # 전체 도서에 대한 판매 만큼 정렬 후 넣기
            most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

            # test 예측값, 이미 구매 했을 경우 제외
            als_predict_list = []
            for user_id in test['customer_id'].unique():
                try:
                    result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                    als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                except:
                    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                    als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                        ]})

            # 100 개만 예측하기
            for idx, pred_list in enumerate(als_predict_list):
                als_predict_list[idx]['items'] = pred_list['items'][:100]
                
            # ALS 
            evaluator = CustomEvaluator()
            ndcg = evaluator._eval(ground_trues, als_predict_list)
            
            als_mf_hyper_parameter['factor'].append(factor)
            als_mf_hyper_parameter['regularization'].append(regularization)
            als_mf_hyper_parameter['iteration'].append(iteration)
            als_mf_hyper_parameter['NDCG'].append(ndcg)

In [70]:
pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False).head()

,factor,regularization,iteration,NDCG
16,10,0.010,5,0.154827
24,10,0.005,5,0.106191
58,20,0.005,15,0.100984
50,20,0.010,15,0.100044
32,15,0.010,5,0.092288


## 7-2. LMF hypter parameter tuning

In [71]:
lmf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[]}

factors = [5,10,15,20]
regularizations = [0.01,0.005]
iterations = [5,10,15,20,25,30,40,50]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
            lmf_model.fit(purchase_sparse, show_progress=False)
            
            # 신규 유저 mp로 넣기
            most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

            # test 예측값
            lmf_predict_list = []
            for user_id in test['customer_id'].unique():
                try:
                    result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                    lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                except:
                    train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                    lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                        if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                        ]})

            # 100 개만 예측하기
            for idx, pred_list in enumerate(lmf_predict_list):
                lmf_predict_list[idx]['items'] = pred_list['items'][:100]
                
            # LMF
            evaluator = CustomEvaluator()
            ndcg = evaluator._eval(ground_trues, lmf_predict_list)
            
            lmf_hyper_parameter['factor'].append(factor)
            lmf_hyper_parameter['regularization'].append(regularization)
            lmf_hyper_parameter['iteration'].append(iteration)
            lmf_hyper_parameter['NDCG'].append(ndcg)

In [72]:
pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False).head()

,factor,regularization,iteration,NDCG
32,15,0.010,5,0.240090
40,15,0.005,5,0.193426
7,5,0.010,50,0.139994
15,5,0.005,50,0.139994
26,10,0.005,15,0.139785


# 8. 결론

- als mf : 0.200100 (factor: 10, regularization: 0.005, iteration: 15)
- lmf : 0.206777    (factor: 15, regularization: 0.005, iteration: 25)
- mp : 0.184859     

- test 신규 아이템을 포함하여 예측을 진행했다고 하더라도 나쁘지 않은 점수가 나옴

# 9. 추천된 items 확인

In [73]:
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 191559.69it/s]


In [74]:
# pred_item, rea_item 비교
def pred_real_dataframe(user_num):
    pred_items_names = []
    predict_dict = als_predict_list[user_num]['items']
    for item in predict_dict:
        pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())

    real_items_names = []
    trues_dict = ground_trues[user_num]['items']
    for item in trues_dict:
        real_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())
    return pd.DataFrame({'pred_item':pred_items_names,'real_item':real_items_names})
    

In [75]:
# train_item, pred_item, real_item 비교
def train_pred_items(user_nums):
    train_pred_items_df = pd.DataFrame(columns=['train_item','pred_item'])
    for user_num in range(1,user_nums):
        train_item_names = []
        for idx in grouped_purchased[grouped_purchased['customer_id']==ground_trues[user_num]['id']].product_ids:
            train_item_names.append(products_df[products_df['_id'] == idx].meta_title.unique()[0])

        pred_items_names = []
        predict_dict = als_predict_list[user_num]['items']
        for item in predict_dict:
            pred_items_names.append(products_df[products_df['_id'] == item].meta_title.unique())


        
        train_pred_items_df.loc[user_num,'train_item'] = train_item_names
        train_pred_items_df.loc[user_num,'pred_item'] = pred_items_names
    return train_pred_items_df


In [76]:
# 예측 유저 구매 횟수 확인
# pd.DataFrame(purchase_sparse[1].todense()).T.value_counts()

In [77]:
print('첫날 :',train['date_paid'].min(),'마지막 날:',train['date_paid'].max())
print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
print('______________________________________________________')
print('첫날 :',test['date_paid'].min(),'마지막 날:',test['date_paid'].max())
print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
print('______________________________________________________')
print('train 데이터수:', len(train))
print('______________________________________________________')
print('train 유저수:',len(set(train.customer_id)))
print('______________________________________________________')
print('test 데이터수:',len(test))
print('______________________________________________________')
print('test 유저수:',len(set(test.customer_id)))
print('______________________________________________________')
print('test 신규 유저 수:',len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
print('______________________________________________________')
print('test 신규 아이템 수:',len(set(test_before_preprocess.product_ids.unique())-set(train_before_preprocess.product_ids.unique())))
# test 전처리 진행했을 경우
print('______________________________________________________')
print('원본 test 수:', len(test))
print('______________________________________________________')
print('전처리 진행했을 경우 test 수:', len(if_prepro_test))
print('______________________________________________________')
print('mf')
display(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False).head(2))
print('______________________________________________________')
print('lmf')
display(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False).head(2))
print('______________________________________________________')
print('mp')
evaluator.evaluate(ground_trues, predict_popular_list)
print('______________________________________________________')
display(pd.DataFrame(medistream_predict_score))

첫날 : 2019-09-23 00:25:41.241000+00:00 마지막 날: 2022-09-10 22:39:59.946000+00:00
train 총 기간: 1083 days 22:14:18.705000
______________________________________________________
첫날 : 2022-09-11 13:03:15.786000+00:00 마지막 날: 2022-09-11 20:17:23.892000+00:00
test 총 기간: 0 days 07:14:08.106000
______________________________________________________
train 데이터수: 37831
______________________________________________________
train 유저수: 7406
______________________________________________________
test 데이터수: 4
______________________________________________________
test 유저수: 2
______________________________________________________
test 신규 유저 수: 0
______________________________________________________
test 신규 아이템 수: 0
______________________________________________________
원본 test 수: 4
______________________________________________________
전처리 진행했을 경우 test 수: 4
______________________________________________________
mf


,factor,regularization,iteration,NDCG
16,10,0.010,5,0.154827
24,10,0.005,5,0.106191


______________________________________________________
lmf


,factor,regularization,iteration,NDCG
32,15,0.010,5,0.240090
40,15,0.005,5,0.193426


______________________________________________________
mp
nDCG: 0.425172
______________________________________________________


,medistream_predict,ndcg,cnt
0,medi_popular,0.425172,2
1,latest,0.082848,1
2,oldest,0.000000,0
3,higt_price,0.193426,1
4,low_price,0.072170,1
5,name_sort,0.058004,1
